In [1]:
from astropy.io import fits
from modules.settings import global_mmf_settings as gset
from astropy.coordinates import SkyCoord
from cosmology import cosmo_fn
import bces.bces as bces
%pylab notebook

def return_log_err(gauss_mean,gauss_err,num_samples=20000,ignore_negatives=True):
    logerr=np.zeros_like(gauss_mean)
    for idx, mu in enumerate(gauss_mean):
        x=np.random.normal(mu,gauss_err[idx],num_samples)
        if ignore_negatives:
            while np.any(x<0):
                neg_idx=np.where(x<0)[0]
                x[neg_idx]=np.random.normal(mu,gauss_err[idx],np.size(neg_idx))
        logerr[idx]=np.std(log10(x))
    return logerr

ImportError: No module named modules.settings

In [113]:
outpath="../results/planck_cosmo_cat/"
gset.setup_mmf_config(dataset="planck",outpath=outpath,chmin=100.,xsize=10.,result_midfix="",do_band_pass=True)

# Cosmo sample xray measurements

In [137]:
dtype=[ "RA", "DEC","z","R500","Mg500","Mg500_err_m","Mg500_err_p","TX","TX_err_m","TX_err_p","M500_YX","M500_err_m","M500_err_p"]
dxray=np.loadtxt(gset.mmfset.cosmo_cat_2013_yxray_file)
xsz_cat={}

for idx,ttype in enumerate(dtype):
    xsz_cat[ttype]=dxray[:,idx]
    
c = SkyCoord(xsz_cat["RA"], xsz_cat["DEC"] , frame='icrs', unit='deg')
xsz_cat["GLON"]=c.galactic.l.degree
xsz_cat["GLAT"]=c.galactic.b.degree
xsz_cat["M500_YX_err"]=(xsz_cat["M500_err_m"]+xsz_cat["M500_err_p"])*0.5
xsz_cat["theta500"]=(xsz_cat["R500"]/(cosmo_fn.dA(xsz_cat["z"])*1000.))*180.*60./np.pi

In [138]:
xsz_cat["theta500"]

array([  5.00885433,   8.49229225,   6.90799487,   9.83207264,
         5.60609699,   3.66965151,   7.71766409,   6.50190334,
         5.92333792,   6.68913825,   4.40404458,   4.22736936,
         7.258689  ,   5.84235404,   5.57155662,   4.19362628,
         6.67206217,   8.22932018,   4.74942679,   4.73343148,
         6.63641534,   4.76375654,   4.79181817,   5.7953465 ,
         4.78384504,   5.02860849,  11.7200855 ,  11.98940349,
         7.25835181,  10.14146942,  11.09439941,  10.07719952,
         4.94756671,   6.76414358,  12.83884795,   7.2071732 ,
         7.69694407,   7.42078057,   6.63365   ,   8.26768282,
         7.71160816,   3.89947283,  14.12178802,  17.49294994,
        15.74966777,  13.85569799,  19.88613877,  21.15959923,
        18.7970216 ,  13.28187614,  16.18033348,  19.37385702,
        22.62754154,  19.80725572,  17.52771784,  15.60654936,
        15.5971715 ,  21.46572663,   6.28774449,   4.80406188,
         5.58157993,   4.87822877,   4.40809286,   3.89

# Cosmo sample SZ measurements

In [114]:
fle=fits.open(gset.mmfset.cosmo_cat_2013_ysz_file)
keys=fle['MY_4_scaling'].header.keys()
fkeys=[k for k in keys if "TTYPE" in k]
fields=[fle['MY_4_scaling'].header[tkeys] for tkeys in fkeys]

ysz_cat={}
for n in fields:
    if n=="NAME":
        ysz_cat[n]=list(fle['MY_4_scaling'].data.field(n).flatten())
    else:
        ysz_cat[n]=fle['MY_4_scaling'].data.field(n).flatten()
        
ysz_cat["GLON"]=np.zeros_like(ysz_cat["REDSHIFT"],dtype=float64)
ysz_cat["GLAT"]=np.zeros_like(ysz_cat["REDSHIFT"],dtype=float64)

for idx in range(np.size(ysz_cat["REDSHIFT"])):
    ysz_cat["GLON"][idx]=float(ysz_cat["NAME"][idx][6:12])
    ysz_cat["GLAT"][idx]=float(ysz_cat["NAME"][idx][12:18])

In [120]:
ysz_cat["Y_500"]
ysz_cat["ERR_Y_500"]

array([  1.40000002e-05,   8.09999983e-06,   6.70000009e-06,
         5.29999988e-06,   9.29999987e-06,   1.70000003e-05,
         7.39999996e-06,   1.10000001e-05,   9.00000032e-06,
         4.39999985e-06,   1.29999999e-05,   9.10000017e-06,
         8.79999970e-06,   9.20000002e-06,   7.90000013e-06,
         1.20000004e-05,   1.20000004e-05,   6.79999994e-06,
         1.10000001e-05,   1.10000001e-05,   6.10000006e-06,
         1.10000001e-05,   7.90000013e-06,   1.29999999e-05,
         1.49999996e-05,   8.40000030e-06,   4.09999984e-06,
         4.20000015e-06,   9.99999975e-06,   3.00000011e-06,
         4.09999984e-06,   3.29999989e-06,   1.40000002e-05,
         9.99999975e-06,   2.30000001e-06,   4.90000002e-06,
         5.40000019e-06,   5.70000020e-06,   5.20000003e-06,
         6.40000007e-06,   6.30000022e-06,   2.09999998e-05,
         3.50000005e-06,   3.00000011e-06,   4.69999986e-06,
         3.59999990e-06,   2.79999995e-06,   3.20000004e-06,
         2.10000007e-06,

# With correction for Malmquist bias

$\alpha =\beta$


In [135]:
Ezgamma=cosmo_fn.Ez(ysz_cat["REDSHIFT"])**(-2./3.)/1e-4
#xdata=log10(ysz_cat["M_Yx_500"]/6e14)
xdata=log10(xsz_cat["M500_YX"]/6e14)
#xerr=return_log_err(ysz_cat["M_Yx_500"]/6e14,ysz_cat["M_Yx_500_err"]/6e14)
xerr=return_log_err(xsz_cat["M500_YX"]/6e14,xsz_cat["M500_YX_err"]/6e14)
# Catalogue
ydata=log10(ysz_cat["Y_500"]*Ezgamma/ysz_cat["BIAS"])
yerr=return_log_err(ysz_cat["Y_500"]*Ezgamma,ysz_cat["ERR_Y_500"]*Ezgamma)
acat,bcat,erracat,errbcat,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print round(acat[3],3),round(erracat[3],3),round(bcat[3],3),round(errbcat[3],3)

BCES, 100000 trials... 
13.533218 s
1.787 0.063 -0.186 0.011


# Without Malmquist bias correction.
- Reproduces the scaling law in Eq. A.6 of Planck 2013 cosmology paper.

In [136]:
Ezgamma=cosmo_fn.Ez(ysz_cat["REDSHIFT"])**(-2./3.)/1e-4
#xdata=log10(ysz_cat["M_Yx_500"]/6e14)
xdata=log10(xsz_cat["M500_YX"]/6e14)
#xerr=return_log_err(ysz_cat["M_Yx_500"]/6e14,ysz_cat["M_Yx_500_err"]/6e14)
xerr=return_log_err(xsz_cat["M500_YX"]/6e14,xsz_cat["M500_YX_err"]/6e14)
# Catalogue
ydata=log10(ysz_cat["Y_500"]*Ezgamma)
yerr=return_log_err(ysz_cat["Y_500"]*Ezgamma,ysz_cat["ERR_Y_500"]*Ezgamma)
acat,bcat,erracat,errbcat,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print round(acat[3],3),round(erracat[3],3),round(bcat[3],3),round(errbcat[3],3)

BCES, 100000 trials... 
13.451791 s
1.768 0.064 -0.175 0.011
